In [16]:
import numpy as np # jsut  in case

In [73]:
#class definition

class TreeNode:
    
    def __init__(self,mass=1,coord=np.zeros([2]),level=0):
        self.mass=mass
        self.coord=coord
        self.children=[]
        self.level=level
        
        
    def addChild(self,child):
        self.children.append(child)
        
    def recomputeValues(self):
        if(len(self.children)>0):
            [self.children[k].recomputeValues for k in np.arange(len(self.children))]#faster
            self.mass = 0
            self.coord = -1
            for child in self.children:
                    try:
                        if self.coord.shape:#if it is not -1
                            self.mass = self.mass + child.mass #really????
                            self.coord = self.coord+ child.coord*child.mass
                    except:
                        self.mass = child.mass
                        self.coord = child.coord * child.mass
                        
            
            self.coord = self.coord / self.mass
        else:
            return self.coord, self.mass
        

In [74]:
def DeepTraverse(root):
    
    if(len(root.children)==0):
        return
    
    for c in root.children:
        print(c.level,c.mass,c.coord)
        DeepTraverse(c)
        

In [75]:
# Try more sophisticated structure

# root ----- p1 ---- p11, p12
#      ----- p2 ---- p21 --- p211,p212,p213

treeRoot = TreeNode()

treeRoot.addChild(TreeNode(mass=1,coord=[0,0],level=1)) #p1
treeRoot.addChild(TreeNode(mass=2,coord=[0,0],level=1)) #p2

treeRoot.children[0].addChild(TreeNode(mass=1.1,coord=[1,1],level=2))
treeRoot.children[0].addChild(TreeNode(mass=1.2,coord=[1,2],level=2))

treeRoot.children[1].addChild(TreeNode(mass=2.1,coord=[0,0],level=2))
treeRoot.children[1].children[0].addChild(TreeNode(mass=2.11,coord=[2,11],level=3))
treeRoot.children[1].children[0].addChild(TreeNode(mass=2.12,coord=[2,12],level=3))
treeRoot.children[1].children[0].addChild(TreeNode(mass=2.13,coord=[2,13],level=3))

DeepTraverse(treeRoot)

1 1 [0, 0]
2 1.1 [1, 1]
2 1.2 [1, 2]
1 2 [0, 0]
2 2.1 [0, 0]
3 2.11 [2, 11]
3 2.12 [2, 12]
3 2.13 [2, 13]


In [76]:
# Tree Construction

def AssignParticlesToClusters(locs,seps):
    #locs: d X N locations
    #seps: N separators
    clIds = np.zeros([locs.shape[1]])
    for locId in np.arange(locs.shape[1]):
        ind=(locs[:,locId]-seps >=0).astype('int32')
        ind=str(ind.tolist())[1:-1].replace(' ','')
        ind=ind.replace(',','')
        clIds[locId]=int( ind, base=2)
        
    return clIds
    
def ConstructTree(xs,ms,root,level):
    #xs is array d X N, N is number of particles, d is dimension
    
    
    if(len(xs.shape)==1 or xs.shape[1]==1):
        #1 particle, leave it
        root.mass = ms
        root.coord = xs
        root.level = level
        
        return root
    
    #Otherwise go recursively
    
    xMin = np.amin(xs,axis=1)
    xMax = np.amax(xs,axis=1)
    separators = (xMin + xMax)/2 # QuadTree
    clusters = AssignParticlesToClusters(xs,separators)
    
    clIds = np.unique(clusters)
    print('AA')
    for clId in clIds:
        clKeys = (clusters==clId)
        child = TreeNode(level=level+1)
        child=ConstructTree(xs[:,clKeys],ms[clKeys],child,level+1)
        child.recomputeValues()
        root.addChild(child)
    
    return root
    


In [77]:
xs = np.array([[0,1,0,1,1.2,1.25],[0,0.75,1,1,1,0.9]])
ms= np.array([1,2,3,4,5,6])

treeRoot = TreeNode()
treeRoot=ConstructTree(xs,ms,treeRoot,level=0)

AA
AA
AA


In [79]:
DeepTraverse(treeRoot)

1 [1] [[0.]
 [0.]]
1 [3] [[0.]
 [1.]]
1 [17] [[1.14705882]
 [0.93529412]]
2 [2] [[1.  ]
 [0.75]]
2 [4] [[1.]
 [1.]]
2 [11] [[1.22727273]
 [0.94545455]]
3 [5] [[1.2]
 [1. ]]
3 [6] [[1.25]
 [0.9 ]]


In [80]:
# ALL IS WORKING!!!!
import quadtrees as qtrees

In [84]:
xs = np.array([[0,1,0,1,1.2,1.25],[0,0.75,1,1,1,0.9]])
ms= np.array([1,2,3,4,5,6])

treeRoot = qtrees.TreeNode()#root
treeRoot=qtrees.ConstructTree(xs,ms,treeRoot,level=0)

AA
AA
AA


In [85]:
DeepTraverse(treeRoot)

1 [1] [[0.]
 [0.]]
1 [3] [[0.]
 [1.]]
1 [17] [[1.14705882]
 [0.93529412]]
2 [2] [[1.  ]
 [0.75]]
2 [4] [[1.]
 [1.]]
2 [11] [[1.22727273]
 [0.94545455]]
3 [5] [[1.2]
 [1. ]]
3 [6] [[1.25]
 [0.9 ]]


In [91]:
treeRoot.recomputeValues


1